In [1]:
!pip install playwright

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 MB 19.7 MB/s eta 0:00:00


In [2]:
!playwright install

167.7 MiB [] 0% 0.0s167.7 MiB [] 0% 296.0s167.7 MiB [] 0% 199.8s167.7 MiB [] 0% 303.3s167.7 MiB [] 0% 194.4s167.7 MiB [] 0% 177.7s167.7 MiB [] 0% 138.8s167.7 MiB [] 0% 105.5s167.7 MiB [] 0% 107.6s167.7 MiB [] 0% 82.1s167.7 MiB [] 0% 65.5s167.7 MiB [] 0% 66.4s167.7 MiB [] 0% 51.0s167.7 MiB [] 0% 39.7s167.7 MiB [] 0% 39.1s167.7 MiB [] 1% 30.2s167.7 MiB [] 1% 23.6s167.7 MiB [] 1% 22.8s167.7 MiB [] 2% 17.7s167.7 MiB [] 3% 13.4s167.7 MiB [] 3% 12.9s167.7 MiB [] 4% 10.3s167.7 MiB [] 5% 9.0s167.7 MiB [] 5% 8.3s167.7 MiB [] 5% 8.4s167.7 MiB [] 6% 7.3s167.7 MiB [] 7% 6.7s167.7 MiB [] 8% 6.3s167.7 MiB [] 9% 5.8s167.7 MiB [] 9% 5.9s167.7 MiB [] 10% 5.6s167.7 MiB [] 10% 5.3s167.7 MiB [] 11% 5.0s167.7 MiB [] 12% 4.8s167.7 MiB [] 13% 4.6s167.7 MiB [] 13% 4.5s167.7 MiB [] 14% 4.4s167.7 MiB [] 15% 4.2s167.7 MiB [] 15% 4.1s167.7 MiB [] 16% 3.9s167.7 MiB [] 17% 3.8s167.7 MiB [] 18% 3.8s167.7 MiB [] 18% 3.6s167.7 MiB [] 19% 3.5s167.7 MiB [] 20% 3.5s167.7 MiB [] 21% 3.4s167.7 MiB [] 22% 3.2s167.7 MiB [] 2

In [ ]:
#!/usr/bin/env python3
import pandas as pd
import re
import time
import os
import urllib.parse
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeoutError
import asyncio
import nest_asyncio

# Apply nest_asyncio to allow running async code in environments with existing event loops
nest_asyncio.apply()

# Function to process HS_CODE and extract Tax ID
def extract_tax_id(hs_code):
    if not isinstance(hs_code, str):
        return None
    match = re.match(r'^"(\d+)\.(\d+)"$', hs_code)
    if match:
        digits = match.group(1) + match.group(2)
        if len(digits) >= 4:
            return digits
    return None

# Async function to scrape VAT rate for a given tax ID
async def scrape_vat_rate(page, tax_id, debug_mode=False):
    vat_rate = "N/A" # Default value
    detail_page_html_path = f'/home/ubuntu/detail_page_{tax_id}.html'
    screenshot_path = f'/home/ubuntu/error_{tax_id}.png'
    results_page_html_path = f'/home/ubuntu/results_page_{tax_id}.html'

    try:
        print(f"Processing Tax ID: {tax_id}")
        # Navigate to the main query page
        await page.goto("https://online.customs.gov.cn/ociswebserver/pages/jckspsl/", wait_until="networkidle", timeout=60000)

        # Input Tax ID
        tax_id_input_selector = "input#dutySpa"
        await page.locator(tax_id_input_selector).wait_for(state="visible", timeout=15000)
        await page.locator(tax_id_input_selector).fill(tax_id)
        print(f"Filled Tax ID: {tax_id}")

        # Click Query button
        query_button_selector = "button#queryBtn"
        query_button = page.locator(query_button_selector)
        await query_button.wait_for(state="visible", timeout=10000)
        if not await query_button.is_disabled(timeout=5000):
            await query_button.click()
            print("Clicked Query button")
        else:
            raise PlaywrightTimeoutError("Query button was visible but disabled")

        # Wait for results
        results_table_selector = "div#result[style*='display: block'] table#listData"
        no_results_table_selector = "div#noResult[style*='display: block']"
        combined_wait_selector = f"{results_table_selector}, {no_results_table_selector}"
        print(f"Waiting for results or no results indicator: {combined_wait_selector}")
        await page.wait_for_selector(combined_wait_selector, state="visible", timeout=25000)
        print("Results or no results indicator appeared.")

        # Save results page HTML for debugging
        try:
            results_html = await page.content()
            with open(results_page_html_path, 'w', encoding='utf-8') as f:
                f.write(results_html)
            print(f"Saved results page HTML to {results_page_html_path}")
        except Exception as html_ex:
            print(f"Could not save results page HTML: {html_ex}")

        navigated_to_detail = False
        # Check if no results
        if await page.locator(no_results_table_selector).is_visible():
            print(f"No data found for Tax ID {tax_id} (No results table displayed).")
            return "No data found"
        # Check for empty results message
        elif await page.locator(f"{results_table_selector} div.ant-empty-description:has-text('暂无数据')").is_visible(timeout=1000):
            print(f"No data found for Tax ID {tax_id} (Ant Design empty message).")
            return "No data found"
        # Check if results exist
        elif await page.locator(results_table_selector).is_visible():
            print("Results table is visible. Extracting detail URL from link attribute.")
            link_anchor_selector = f"{results_table_selector} td a.showDetailA"
            try:
                link_locator = page.locator(link_anchor_selector).first
                await link_locator.wait_for(state="visible", timeout=10000)
                name_attribute = await link_locator.get_attribute("name")
                if name_attribute:
                    print(f"Found name attribute: {name_attribute[:100]}...")
                    base_detail_url = "https://online.customs.gov.cn/ociswebserver/pages/jckspsl/detail.html"
                    detail_url = f"{base_detail_url}?id={name_attribute}"
                    print(f"Constructed detail URL: {detail_url}")

                    await page.goto(detail_url, wait_until="networkidle", timeout=60000)
                    print(f"Navigated directly to detail page: {page.url}")
                    navigated_to_detail = True
                else:
                    print("Could not find 'name' attribute on the link.")
                    raise Exception("Link found, but 'name' attribute missing.")
            except PlaywrightTimeoutError as e_link_attr:
                print(f"Could not find the 'More tax rates' link or its attribute: {e_link_attr}")
                raise Exception("Results table found, but failed to find link or extract detail URL.")
        else:
            raise Exception("Could not determine if results or no results were displayed.")

        if not navigated_to_detail:
            raise Exception("Failed to navigate to details page.")

        current_url = page.url
        print(f"Current URL after navigation: {current_url}")
        if "detail.html" not in current_url:
            print("Error: Did not navigate to detail page!")
            await page.screenshot(path=screenshot_path)
            print(f"Screenshot saved to {screenshot_path}")
            raise Exception("Navigation to detail page failed")

        print("Successfully on detail page")
        # Save detail page for debugging
        try:
            detail_html = await page.content()
            with open(detail_page_html_path, 'w', encoding='utf-8') as f:
                f.write(detail_html)
            print(f"Saved detail page HTML to {detail_page_html_path}")
            await page.screenshot(path=f'/home/ubuntu/detail_page_screenshot_{tax_id}.png')
            print(f"Saved detail page screenshot to /home/ubuntu/detail_page_screenshot_{tax_id}.png")
        except Exception as html_ex:
            print(f"Could not save detail page HTML/Screenshot: {html_ex}")

        if debug_mode:
            print("Debug mode: Stopping after saving detail page content.")
            return "Debug Stop"

        # VAT Extraction Logic
        try:
            expand_link_selector = "a:has-text('进口消费税税率、增值税税率')"
            expand_link = page.locator(expand_link_selector).first
            await expand_link.wait_for(state="visible", timeout=15000)
            print("Found VAT section link. Clicking to expand/load...")
            await expand_link.click()

            vat_table_content_selector = "table.showTable tbody#add tr"
            await page.locator(vat_table_content_selector).first.wait_for(state="visible", timeout=20000)
            print("VAT table content appeared.")

            vat_rate_selector = f"{vat_table_content_selector} td:nth-child(2)"
            vat_cell = page.locator(vat_rate_selector).first
            vat_rate = await vat_cell.text_content()
            vat_rate = vat_rate.strip() if vat_rate else "N/A"
            print(f"Found VAT Rate: {vat_rate}")

            # Clean up debug files if successful
            if os.path.exists(detail_page_html_path):
                try: os.remove(detail_page_html_path)
                except: pass
            if os.path.exists(results_page_html_path):
                try: os.remove(results_page_html_path)
                except: pass

        except PlaywrightTimeoutError as e_vat_extract:
            print(f"Error during VAT extraction on detail page: {e_vat_extract}")
            vat_rate = "Error: Rate extraction failed"
            try:
                await page.screenshot(path=f'/home/ubuntu/error_detail_extract_{tax_id}.png')
                print(f"Detail extraction error screenshot saved to /home/ubuntu/error_detail_extract_{tax_id}.png")
            except Exception as screenshot_error:
                print(f"Could not save detail extraction error screenshot: {screenshot_error}")

    except Exception as e:
        print(f"Error processing Tax ID {tax_id}: {e}")
        try:
            if not os.path.exists(f'/home/ubuntu/error_detail_extract_{tax_id}.png'):
                await page.screenshot(path=screenshot_path)
                print(f"Screenshot saved to {screenshot_path}")
        except Exception as screenshot_error:
            print(f"Could not save screenshot: {screenshot_error}")
        if vat_rate == "N/A" and "No data found" not in str(e):
            vat_rate = "Error"

    return vat_rate

async def process_tax_ids(valid_tax_ids, run_in_debug_mode=False):
    vat_rates_map = {}
    async with async_playwright() as p:
        browser = None
        try:
            browser = await p.chromium.launch(headless=True)
            print("Launched Chromium browser")
        except Exception as e_chromium:
            print(f"Failed to launch Chromium: {e_chromium}. Trying Firefox.")
            try:
                browser = await p.firefox.launch(headless=True)
                print("Launched Firefox browser")
            except Exception as e_firefox:
                print(f"Failed to launch Firefox: {e_firefox}. Exiting.")
                exit()

        if browser:
            page = await browser.new_page()
            for tax_id in valid_tax_ids:
                rate = await scrape_vat_rate(page, tax_id, debug_mode=run_in_debug_mode)
                vat_rates_map[tax_id] = rate
                await asyncio.sleep(2)  # Delay between requests

            await browser.close()
            print("Browser closed.")
        else:
            print("Failed to launch any browser.")
            exit()
    return vat_rates_map

def run_async_code(valid_tax_ids, run_in_debug_mode=False):
    try:
        # Try to get the running loop first
        loop = asyncio.get_running_loop()
    except RuntimeError:  # No running event loop
        loop = None

    if loop:  # We're in a notebook or async environment
        print("Running in existing event loop")
        return loop.run_until_complete(process_tax_ids(valid_tax_ids, run_in_debug_mode))
    else:  # Normal script execution
        print("Running in new event loop")
        return asyncio.run(process_tax_ids(valid_tax_ids, run_in_debug_mode))

if __name__ == "__main__":
    input_csv_path = "/content/clean_V7_1_Rate_without_footnotes.csv"
    output_csv_path = "/content/clean_V7_1_1_with_vat.csv"

    try:
        df = pd.read_csv(input_csv_path)
        print(f"Successfully loaded CSV: {input_csv_path}")
    except Exception as e:
        print(f"Error loading CSV {input_csv_path}: {e}")
        exit()

    if 'VAT_Rate' not in df.columns:
        df['VAT_Rate'] = "N/A"

    df['Tax_ID'] = df['HS_CODE'].apply(extract_tax_id)
    valid_tax_ids = df['Tax_ID'].dropna().unique().tolist()
    print(f"Found {len(valid_tax_ids)} unique valid Tax IDs to process.")

    run_in_debug_mode = False

    # Run the async code with proper event loop handling
    vat_rates_map = run_async_code(valid_tax_ids, run_in_debug_mode)

    print("\n--- Run Results ---")
    for tax_id, rate in vat_rates_map.items():
        print(f"Tax ID: {tax_id}, Scraped VAT Rate: {rate}")
    print("-------------------\n")

    if not run_in_debug_mode and vat_rates_map:
        vat_series = pd.Series(vat_rates_map)
        mask = df['Tax_ID'].isin(vat_series.index) & (~df['VAT_Rate'].isin(["Error", "No data found", "Error: Rate extraction failed"]))
        df.loc[mask, 'VAT_Rate'] = df.loc[mask, 'Tax_ID'].map(vat_series)

        try:
            df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
            print(f"Successfully saved updated CSV to: {output_csv_path}")
        except Exception as e:
            print(f"Error saving updated CSV {output_csv_path}: {e}")
    elif run_in_debug_mode:
        print("Debug mode was active. No CSV file saved.")
    else:
        print("No results generated or error occurred. No CSV file saved.")

    print("Script finished.")

Successfully loaded CSV: /content/clean_V7_1_Rate_without_footnotes.csv
Found 10138 unique valid Tax IDs to process.
Running in existing event loop
Processing Tax ID: 03028400
Error processing Tax ID 03028400: Page.goto: Connection closed while reading from the driver
Could not save screenshot: Page.screenshot: Connection closed while reading from the driver


Processing Tax ID: 1007
Error processing Tax ID 1007: Page.goto: Connection closed while reading from the driver
Could not save screenshot: Page.screenshot: Connection closed while reading from the driver
Launched Chromium browser
Processing Tax ID: 0101


Processing Tax ID: 03028500
Error processing Tax ID 03028500: Page.goto: Connection closed while reading from the driver
Could not save screenshot: Page.screenshot: Connection closed while reading from the driver


Processing Tax ID: 10071000
Error processing Tax ID 10071000: Page.goto: Connection closed while reading from the driver
Could not save screenshot: Page.screenshot: Connection closed while reading from the driver
Filled Tax ID: 0101
Clicked Query button
Waiting for results or no results indicator: div#result[style*='display: block'] table#listData, div#noResult[style*='display: block']
Results or no results indicator appeared.
Could not save results page HTML: [Errno 2] No such file or directory: '/home/ubuntu/results_page_0101.html'
Results table is visible. Extracting detail URL from link attribute.
Found name attribute: %257B%2522gName%2522:%2522%25E6%2594%25B9%25E8%2589%25AF%25E7%25A7%258D%25E7%2594%25A8%25E6%25BF%259...
Constructed detail URL: https://online.customs.gov.cn/ociswebserver/pages/jckspsl/detail.html?id=%257B%2522gName%2522:%2522%25E6%2594%25B9%25E8%2589%25AF%25E7%25A7%258D%25E7%2594%25A8%25E6%25BF%2592%25E5%258D%25B1%25E9%2587%258E%25E9%25A9%25AC%2522,%2522codeTs%2522

Processing Tax ID: 03028910
Error processing Tax ID 03028910: Page.goto: Connection closed while reading from the driver
Could not save screenshot: Page.screenshot: Connection closed while reading from the driver


...

Processing Tax ID: 03044200
Error processing Tax ID 03044200: Page.goto: Connection closed while reading from the driver
Could not save screenshot: Page.screenshot: Connection closed while reading from the driver
Filled Tax ID: 01051490


Clicked Query button
Waiting for results or no results indicator: div#result[style*='display: block'] table#listData, div#noResult[style*='display: block']
Processing Tax ID: 12024200
Error processing Tax ID 12024200: Page.goto: Connection closed while reading from the driver
Could not save screenshot: Page.screenshot: Connection closed while reading from the driver
Results or no results indicator appeared.
Could not save results page HTML: [Errno 2] No such file or directory: '/home/ubuntu/results_page_01051490.html'
Results table is visible. Extracting detail URL from link attribute.
Found name attribute: %257B%2522gName%2522:%2522%25E4%25B8%258D%25E8%25B6%2585%25E8%25BF%2587185%25E5%2585%258B%25E7%259A%...
Constructed detail URL: https://online.customs.gov.cn/ociswebserver/pages/jckspsl/detail.html?id=%257B%2522gName%2522:%2522%25E4%25B8%258D%25E8%25B6%2585%25E8%25BF%2587185%25E5%2585%258B%25E7%259A%2584%25E5%2585%25B6%25E4%25BB%2596%25E9%25B9%2585%2522,%2522codeTs%2522:%25220105149

Processing Tax ID: 03044300
Error processing Tax ID 03044300: Page.goto: Connection closed while reading from the driver
Could not save screenshot: Page.screenshot: Connection closed while reading from the driver


Processing Tax ID: 1203
Error processing Tax ID 1203: Page.goto: Connection closed while reading from the driver
Could not save screenshot: Page.screenshot: Connection closed while reading from the driver


KeyboardInterrupt: 

In [ ]:
#!/usr/bin/env python3
import pandas as pd
import re
import time
import os
import urllib.parse
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeoutError
import asyncio
import nest_asyncio

# Apply nest_asyncio to allow running async code in environments with existing event loops
nest_asyncio.apply()

def extract_tax_id(hs_code):
    if not isinstance(hs_code, str):
        return None
    match = re.match(r'^"(\d+)\.(\d+)"$', hs_code)
    if match:
        digits = match.group(1) + match.group(2)
        if len(digits) >= 4:
            return digits
    return None

async def scrape_vat_rates_from_detail_page(page, tax_id, debug_mode=False):
    vat_rate = "N/A"
    detail_page_html_path = f'/home/ubuntu/detail_page_{tax_id}.html'
    screenshot_path = f'/home/ubuntu/error_{tax_id}.png'

    try:
        print(f"Processing VAT rate for Tax ID: {tax_id}")

        # VAT Extraction Logic
        try:
            expand_link_selector = "a:has-text('进口消费税税率、增值税税率')"
            expand_link = page.locator(expand_link_selector).first
            await expand_link.wait_for(state="visible", timeout=15000)
            print("Found VAT section link. Clicking to expand/load...")
            await expand_link.click()

            vat_table_content_selector = "table.showTable tbody#add tr"
            await page.locator(vat_table_content_selector).first.wait_for(state="visible", timeout=20000)
            print("VAT table content appeared.")

            vat_rate_selector = f"{vat_table_content_selector} td:nth-child(2)"
            vat_cell = page.locator(vat_rate_selector).first
            vat_rate = await vat_cell.text_content()
            vat_rate = vat_rate.strip() if vat_rate else "N/A"
            print(f"Found VAT Rate: {vat_rate}")

        except PlaywrightTimeoutError as e_vat_extract:
            print(f"Error during VAT extraction on detail page: {e_vat_extract}")
            vat_rate = "Error: Rate extraction failed"
            try:
                await page.screenshot(path=f'/home/ubuntu/error_detail_extract_{tax_id}.png')
                print(f"Detail extraction error screenshot saved to /home/ubuntu/error_detail_extract_{tax_id}.png")
            except Exception as screenshot_error:
                print(f"Could not save detail extraction error screenshot: {screenshot_error}")

    except Exception as e:
        print(f"Error processing Tax ID {tax_id}: {e}")
        try:
            if not os.path.exists(f'/home/ubuntu/error_detail_extract_{tax_id}.png'):
                await page.screenshot(path=screenshot_path)
                print(f"Screenshot saved to {screenshot_path}")
        except Exception as screenshot_error:
            print(f"Could not save screenshot: {screenshot_error}")
        if vat_rate == "N/A":
            vat_rate = "Error"

    return vat_rate

async def process_results_table(page, original_tax_id, debug_mode=False):
    results = {}
    results_table_selector = "div#result[style*='display: block'] table#listData"

    try:
        # Get all rows in the results table
        rows = await page.locator(f"{results_table_selector} tbody tr").count()
        print(f"Found {rows} tax IDs in results table")

        for i in range(rows):
            try:
                # Get the tax ID from each row (first column)
                tax_id_cell = page.locator(f"{results_table_selector} tbody tr:nth-child({i+1}) td:nth-child(2)")
                tax_id = await tax_id_cell.text_content()
                tax_id = tax_id.strip() if tax_id else None

                if tax_id:
                    print(f"Processing result #{i+1}: {tax_id}")

                    # Get the detail link for this row
                    link_anchor_selector = f"{results_table_selector} tbody tr:nth-child({i+1}) td a.showDetailA"
                    link_locator = page.locator(link_anchor_selector).first
                    await link_locator.wait_for(state="visible", timeout=10000)
                    name_attribute = await link_locator.get_attribute("name")

                    if name_attribute:
                        print(f"Found name attribute for {tax_id}")
                        base_detail_url = "https://online.customs.gov.cn/ociswebserver/pages/jckspsl/detail.html"
                        detail_url = f"{base_detail_url}?id={name_attribute}"
                        print(f"Navigating to detail URL for {tax_id}")

                        # Open detail page in new tab
                        context = page.context
                        new_page = await context.new_page()
                        await new_page.goto(detail_url, wait_until="networkidle", timeout=60000)

                        # Scrape VAT rate from detail page
                        vat_rate = await scrape_vat_rates_from_detail_page(new_page, tax_id, debug_mode)
                        results[tax_id] = vat_rate

                        # Close the detail tab
                        await new_page.close()
                    else:
                        print(f"No detail link found for {tax_id}")
                        results[tax_id] = "Error: No detail link"
            except Exception as e:
                print(f"Error processing row {i+1}: {e}")
                if tax_id:
                    results[tax_id] = "Error: Row processing failed"

    except Exception as e:
        print(f"Error processing results table: {e}")
        results[original_tax_id] = "Error: Results table processing failed"

    return results

async def scrape_vat_rates(page, tax_id, debug_mode=False):
    results_page_html_path = f'/home/ubuntu/results_page_{tax_id}.html'
    screenshot_path = f'/home/ubuntu/error_{tax_id}.png'
    results = {}

    try:
        print(f"Querying Tax ID: {tax_id}")
        await page.goto("https://online.customs.gov.cn/ociswebserver/pages/jckspsl/", wait_until="networkidle", timeout=60000)

        # Input Tax ID
        tax_id_input_selector = "input#dutySpa"
        await page.locator(tax_id_input_selector).wait_for(state="visible", timeout=15000)
        await page.locator(tax_id_input_selector).fill(tax_id)
        print(f"Filled Tax ID: {tax_id}")

        # Click Query button
        query_button_selector = "button#queryBtn"
        query_button = page.locator(query_button_selector)
        await query_button.wait_for(state="visible", timeout=10000)
        if not await query_button.is_disabled(timeout=5000):
            await query_button.click()
            print("Clicked Query button")
        else:
            raise PlaywrightTimeoutError("Query button was visible but disabled")

        # Wait for results
        results_table_selector = "div#result[style*='display: block'] table#listData"
        no_results_table_selector = "div#noResult[style*='display: block']"
        combined_wait_selector = f"{results_table_selector}, {no_results_table_selector}"
        await page.wait_for_selector(combined_wait_selector, state="visible", timeout=25000)
        print("Results appeared")

        # Save results page HTML for debugging
        try:
            results_html = await page.content()
            with open(results_page_html_path, 'w', encoding='utf-8') as f:
                f.write(results_html)
            print(f"Saved results page HTML to {results_page_html_path}")
        except Exception as html_ex:
            print(f"Could not save results page HTML: {html_ex}")

        # Check if no results
        if await page.locator(no_results_table_selector).is_visible():
            print(f"No data found for Tax ID {tax_id}")
            results[tax_id] = "No data found"
        elif await page.locator(f"{results_table_selector} div.ant-empty-description:has-text('暂无数据')").is_visible(timeout=1000):
            print(f"No data found for Tax ID {tax_id}")
            results[tax_id] = "No data found"
        elif await page.locator(results_table_selector).is_visible():
            print("Results table found, processing...")
            results = await process_results_table(page, tax_id, debug_mode)
        else:
            raise Exception("Could not determine results status")

    except Exception as e:
        print(f"Error processing Tax ID {tax_id}: {e}")
        try:
            await page.screenshot(path=screenshot_path)
            print(f"Screenshot saved to {screenshot_path}")
        except Exception as screenshot_error:
            print(f"Could not save screenshot: {screenshot_error}")
        results[tax_id] = "Error: Initial query failed"

    return results

async def process_tax_ids(valid_tax_ids, run_in_debug_mode=False):
    all_results = {}
    async with async_playwright() as p:
        browser = None
        try:
            browser = await p.chromium.launch(headless=True)
            print("Launched Chromium browser")
        except Exception as e_chromium:
            print(f"Failed to launch Chromium: {e_chromium}. Trying Firefox.")
            try:
                browser = await p.firefox.launch(headless=True)
                print("Launched Firefox browser")
            except Exception as e_firefox:
                print(f"Failed to launch Firefox: {e_firefox}. Exiting.")
                return all_results

        if browser:
            context = await browser.new_context()
            page = await context.new_page()

            for tax_id in valid_tax_ids:
                results = await scrape_vat_rates(page, tax_id, debug_mode=run_in_debug_mode)
                all_results.update(results)
                await asyncio.sleep(2)  # Delay between requests

            await context.close()
            await browser.close()
            print("Browser closed.")
        else:
            print("Failed to launch any browser.")

    return all_results

def run_async_code(valid_tax_ids, run_in_debug_mode=False):
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:
        loop = None

    if loop:
        print("Running in existing event loop")
        return loop.run_until_complete(process_tax_ids(valid_tax_ids, run_in_debug_mode))
    else:
        print("Running in new event loop")
        return asyncio.run(process_tax_ids(valid_tax_ids, run_in_debug_mode))

if __name__ == "__main__":
    input_csv_path = "/content/clean_V7_1_Rate_without_footnotes.csv"
    output_csv_path = "/content/clean_V7_1_1_with_vat.csv"

    try:
        df = pd.read_csv(input_csv_path)
        print(f"Successfully loaded CSV: {input_csv_path}")
    except Exception as e:
        print(f"Error loading CSV {input_csv_path}: {e}")
        exit()

    # Extract all unique tax IDs from the input file
    df['Tax_ID'] = df['HS_CODE'].apply(extract_tax_id)
    input_tax_ids = df['Tax_ID'].dropna().unique().tolist()
    print(f"Found {len(input_tax_ids)} unique input Tax IDs to process.")

    run_in_debug_mode = False

    # Run the scraping process
    vat_rates_map = run_async_code(input_tax_ids, run_in_debug_mode)

    print("\n--- Final Results ---")
    for tax_id, rate in vat_rates_map.items():
        print(f"Tax ID: {tax_id}, VAT Rate: {rate}")
    print("-------------------\n")

    # Create a new DataFrame with all discovered tax IDs and their VAT rates
    results_df = pd.DataFrame({
        'Tax_ID': list(vat_rates_map.keys()),
        'VAT_Rate': list(vat_rates_map.values())
    })

    # Save the results to CSV
    try:
        results_df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
        print(f"Successfully saved results to: {output_csv_path}")
    except Exception as e:
        print(f"Error saving results CSV {output_csv_path}: {e}")

    print("Script finished.")

Streaming output truncated to the last 5000 lines.
Found VAT Rate: 9%
Querying Tax ID: 03072210
Filled Tax ID: 03072210
Clicked Query button
Results appeared
Could not save results page HTML: [Errno 2] No such file or directory: '/home/ubuntu/results_page_03072210.html'
Results table found, processing...
Found 1 tax IDs in results table
Processing result #1: 0307221000
Found name attribute for 0307221000
Navigating to detail URL for 0307221000
Processing VAT rate for Tax ID: 0307221000
Found VAT section link. Clicking to expand/load...
VAT table content appeared.
Found VAT Rate: 9%
Querying Tax ID: 03072290
Filled Tax ID: 03072290
Clicked Query button
Results appeared
Could not save results page HTML: [Errno 2] No such file or directory: '/home/ubuntu/results_page_03072290.html'
Results table found, processing...
Found 1 tax IDs in results table
Processing result #1: 0307229000
Found name attribute for 0307229000
Navigating to detail URL for 0307229000
Processing VAT rate for Tax ID: 0

In [3]:
#!/usr/bin/env python3
import pandas as pd
import re
import time
import os
import json
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeoutError
import asyncio
import nest_asyncio
from pathlib import Path

# Apply nest_asyncio to allow running async code in environments with existing event loops
nest_asyncio.apply()

# Constants
RESULTS_FILE = "tax_id_VAT.json"
LOG_FILE = "scrape_progress.log"

def setup_logging():
    """Set up logging to file"""
    with open(LOG_FILE, 'a') as f:
        f.write(f"\n\n=== New Run Started at {time.ctime()} ===\n")

def log_message(message):
    """Log a message with timestamp"""
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
    log_entry = f"[{timestamp}] {message}\n"
    print(log_entry.strip())
    with open(LOG_FILE, 'a') as f:
        f.write(log_entry)

def load_existing_results():
    """Load existing results from file if it exists"""
    if os.path.exists(RESULTS_FILE):
        try:
            with open(RESULTS_FILE, 'r') as f:
                return json.load(f)
        except Exception as e:
            log_message(f"Error loading existing results: {e}")
    return {}

def save_results(results):
    """Save results to file"""
    try:
        with open(RESULTS_FILE, 'w') as f:
            json.dump(results, f, indent=2)
    except Exception as e:
        log_message(f"Error saving results: {e}")

def extract_tax_id(hs_code):
    if not isinstance(hs_code, str):
        return None
    match = re.match(r'^"(\d+)\.(\d+)"$', hs_code)
    if match:
        digits = match.group(1) + match.group(2)
        if len(digits) >= 4:
            return digits
    return None

async def scrape_vat_rates_from_detail_page(page, tax_id, debug_mode=False):
    vat_rate = "N/A"
    detail_page_html_path = f'detail_page_{tax_id}.html'
    screenshot_path = f'error_{tax_id}.png'

    try:
        log_message(f"Processing VAT rate for Tax ID: {tax_id}")

        # VAT Extraction Logic
        try:
            expand_link_selector = "a:has-text('进口消费税税率、增值税税率')"
            expand_link = page.locator(expand_link_selector).first
            await expand_link.wait_for(state="visible", timeout=15000)
            log_message("Found VAT section link. Clicking to expand/load...")
            await expand_link.click()

            vat_table_content_selector = "table.showTable tbody#add tr"
            await page.locator(vat_table_content_selector).first.wait_for(state="visible", timeout=20000)
            log_message("VAT table content appeared.")

            vat_rate_selector = f"{vat_table_content_selector} td:nth-child(2)"
            vat_cell = page.locator(vat_rate_selector).first
            vat_rate = await vat_cell.text_content()
            vat_rate = vat_rate.strip() if vat_rate else "N/A"
            log_message(f"Found VAT Rate: {vat_rate}")

        except PlaywrightTimeoutError as e_vat_extract:
            log_message(f"Error during VAT extraction on detail page: {e_vat_extract}")
            vat_rate = "Error: Rate extraction failed"
            try:
                await page.screenshot(path=screenshot_path)
                log_message(f"Saved error screenshot to {screenshot_path}")
            except Exception as screenshot_error:
                log_message(f"Could not save screenshot: {screenshot_error}")

    except Exception as e:
        log_message(f"Error processing Tax ID {tax_id}: {e}")
        try:
            if not os.path.exists(f'error_detail_extract_{tax_id}.png'):
                await page.screenshot(path=screenshot_path)
                log_message(f"Screenshot saved to {screenshot_path}")
        except Exception as screenshot_error:
            log_message(f"Could not save screenshot: {screenshot_error}")
        if vat_rate == "N/A":
            vat_rate = "Error"

    return vat_rate

async def process_results_table(page, original_tax_id, debug_mode=False):
    results = {}
    results_table_selector = "div#result[style*='display: block'] table#listData"
    existing_results = load_existing_results()  # Load existing results before processing

    try:
        # Get all rows in the results table
        rows = await page.locator(f"{results_table_selector} tbody tr").count()
        log_message(f"Found {rows} tax IDs in results table")

        for i in range(rows):
            tax_id = None
            try:
                # Get the tax ID from each row (second column)
                tax_id_cell = page.locator(f"{results_table_selector} tbody tr:nth-child({i+1}) td:nth-child(2)")
                tax_id = await tax_id_cell.text_content()
                tax_id = tax_id.strip() if tax_id else None

                # Validate tax ID format before processing
                if not tax_id or not tax_id.isdigit() or len(tax_id) < 4:
                    log_message(f"Invalid tax ID format: {tax_id} - Skipping")
                    continue

                # Skip if already successfully processed
                if tax_id in existing_results and existing_results[tax_id] not in ["Error", "No data found"]:
                    log_message(f"Tax ID {tax_id} already processed - Skipping")
                    continue

                log_message(f"Processing result #{i+1}: {tax_id}")

                # Get the detail link for this row
                link_anchor_selector = f"{results_table_selector} tbody tr:nth-child({i+1}) td a.showDetailA"
                link_locator = page.locator(link_anchor_selector).first
                await link_locator.wait_for(state="visible", timeout=10000)
                name_attribute = await link_locator.get_attribute("name")

                if name_attribute:
                    log_message(f"Found name attribute for {tax_id}")
                    base_detail_url = "https://online.customs.gov.cn/ociswebserver/pages/jckspsl/detail.html"
                    detail_url = f"{base_detail_url}?id={name_attribute}"
                    log_message(f"Navigating to detail URL for {tax_id}")

                    # Open detail page in new tab
                    context = page.context
                    new_page = await context.new_page()
                    await new_page.goto(detail_url, wait_until="networkidle", timeout=60000)

                    # Scrape VAT rate from detail page
                    vat_rate = await scrape_vat_rates_from_detail_page(new_page, tax_id, debug_mode)

                    # Only update results if we got a valid VAT rate
                    if vat_rate not in ["N/A", "Error"]:
                        results[tax_id] = vat_rate
                        existing_results[tax_id] = vat_rate  # Update in-memory cache
                        save_results(existing_results)  # Immediate save
                    else:
                        log_message(f"Invalid VAT rate for {tax_id}: {vat_rate}")

                    # Close the detail tab
                    await new_page.close()
                else:
                    log_message(f"No detail link found for {tax_id}")
                    results[tax_id] = "Error: No detail link"

            except Exception as e:
                log_message(f"Error processing row {i+1}: {e}")
                if tax_id:
                    results[tax_id] = f"Error: {str(e)}"
                    # Mark as error in results but don't save to prevent pollution
                    # We'll retry these on next run

    except Exception as e:
        log_message(f"Error processing results table: {e}")
        # Only mark original tax ID as error if we didn't process any results
        if not results:
            results[original_tax_id] = f"Error: Results table processing failed - {str(e)}"

    return results

async def scrape_vat_rates(page, tax_id, debug_mode=False):
    results_page_html_path = f'results_page_{tax_id}.html'
    screenshot_path = f'error_{tax_id}.png'
    results = {}

    try:
        log_message(f"Querying Tax ID: {tax_id}")
        await page.goto("https://online.customs.gov.cn/ociswebserver/pages/jckspsl/", wait_until="networkidle", timeout=60000)

        # Input Tax ID
        tax_id_input_selector = "input#dutySpa"
        await page.locator(tax_id_input_selector).wait_for(state="visible", timeout=15000)
        await page.locator(tax_id_input_selector).fill(tax_id)
        log_message(f"Filled Tax ID: {tax_id}")

        # Click Query button
        query_button_selector = "button#queryBtn"
        query_button = page.locator(query_button_selector)
        await query_button.wait_for(state="visible", timeout=10000)
        if not await query_button.is_disabled(timeout=5000):
            await query_button.click()
            log_message("Clicked Query button")
        else:
            raise PlaywrightTimeoutError("Query button was visible but disabled")

        # Wait for results
        results_table_selector = "div#result[style*='display: block'] table#listData"
        no_results_table_selector = "div#noResult[style*='display: block']"
        combined_wait_selector = f"{results_table_selector}, {no_results_table_selector}"
        await page.wait_for_selector(combined_wait_selector, state="visible", timeout=25000)
        log_message("Results appeared")

        # Check if no results
        if await page.locator(no_results_table_selector).is_visible():
            log_message(f"No data found for Tax ID {tax_id}")
            results[tax_id] = "No data found"
        elif await page.locator(f"{results_table_selector} div.ant-empty-description:has-text('暂无数据')").is_visible(timeout=1000):
            log_message(f"No data found for Tax ID {tax_id}")
            results[tax_id] = "No data found"
        elif await page.locator(results_table_selector).is_visible():
            log_message("Results table found, processing...")
            results = await process_results_table(page, tax_id, debug_mode)
        else:
            raise Exception("Could not determine results status")

    except Exception as e:
        log_message(f"Error processing Tax ID {tax_id}: {e}")
        try:
            await page.screenshot(path=screenshot_path)
            log_message(f"Screenshot saved to {screenshot_path}")
        except Exception as screenshot_error:
            log_message(f"Could not save screenshot: {screenshot_error}")
        results[tax_id] = "Error: Initial query failed"

    return results

async def process_tax_ids(valid_tax_ids, run_in_debug_mode=False):
    all_results = load_existing_results()
    processed_ids = set(all_results.keys())
    remaining_ids = [tid for tid in valid_tax_ids if tid not in processed_ids]

    log_message(f"Resuming scraping. Already processed: {len(processed_ids)}, Remaining: {len(remaining_ids)}")

    async with async_playwright() as p:
        browser = None
        try:
            browser = await p.chromium.launch(headless=True)
            log_message("Launched Chromium browser")
        except Exception as e_chromium:
            log_message(f"Failed to launch Chromium: {e_chromium}. Trying Firefox.")
            try:
                browser = await p.firefox.launch(headless=True)
                log_message("Launched Firefox browser")
            except Exception as e_firefox:
                log_message(f"Failed to launch Firefox: {e_firefox}. Exiting.")
                return all_results

        if browser:
            context = await browser.new_context()
            page = await context.new_page()

            for i, tax_id in enumerate(remaining_ids, 1):
                try:
                    log_message(f"Processing {i}/{len(remaining_ids)}: {tax_id}")
                    results = await scrape_vat_rates(page, tax_id, run_in_debug_mode)
                    all_results.update(results)

                    # Save progress after each tax ID
                    save_results(all_results)

                    # Add delay between requests
                    await asyncio.sleep(2)

                except Exception as e:
                    log_message(f"Fatal error processing {tax_id}: {e}")
                    # Save what we have before exiting
                    save_results(all_results)
                    break

            await context.close()
            await browser.close()
            log_message("Browser closed.")
        else:
            log_message("Failed to launch any browser.")

    return all_results

def run_async_code(valid_tax_ids, run_in_debug_mode=False):
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:
        loop = None

    if loop:
        log_message("Running in existing event loop")
        return loop.run_until_complete(process_tax_ids(valid_tax_ids, run_in_debug_mode))
    else:
        log_message("Running in new event loop")
        return asyncio.run(process_tax_ids(valid_tax_ids, run_in_debug_mode))

if __name__ == "__main__":
    setup_logging()

    input_csv_path = "clean_V7_1_Rate_without_footnotes.csv"
    output_csv_path = "clean_V7_1_1_with_vat.csv"

    try:
        df = pd.read_csv(input_csv_path)
        log_message(f"Successfully loaded CSV: {input_csv_path}")
    except Exception as e:
        log_message(f"Error loading CSV {input_csv_path}: {e}")
        exit()

    # Extract all unique tax IDs from the input file
    df['Tax_ID'] = df['HS_CODE'].apply(extract_tax_id)
    input_tax_ids = df['Tax_ID'].dropna().unique().tolist()
    log_message(f"Found {len(input_tax_ids)} unique input Tax IDs to process.")

    run_in_debug_mode = False

    # Run the scraping process
    vat_rates_map = run_async_code(input_tax_ids, run_in_debug_mode)

    log_message("\n--- Final Results ---")
    for tax_id, rate in vat_rates_map.items():
        log_message(f"Tax ID: {tax_id}, VAT Rate: {rate}")
    log_message("-------------------\n")

    # Create a new DataFrame with all discovered tax IDs and their VAT rates
    results_df = pd.DataFrame({
        'Tax_ID': list(vat_rates_map.keys()),
        'VAT_Rate': list(vat_rates_map.values())
    })

    # Save the results to CSV
    try:
        results_df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
        log_message(f"Successfully saved results to: {output_csv_path}")
    except Exception as e:
        log_message(f"Error saving results CSV {output_csv_path}: {e}")

    log_message("Script finished.")

Streaming output truncated to the last 5000 lines.
[2025-06-06 14:56:43] Tax ID: 6802921000, VAT Rate: 13%
[2025-06-06 14:56:43] Tax ID: 6802929000, VAT Rate: 13%
[2025-06-06 14:56:43] Tax ID: 6802931100, VAT Rate: 13%
[2025-06-06 14:56:43] Tax ID: 6802931900, VAT Rate: 13%
[2025-06-06 14:56:43] Tax ID: 6802939000, VAT Rate: 13%
[2025-06-06 14:56:43] Tax ID: 6802991000, VAT Rate: 13%
[2025-06-06 14:56:43] Tax ID: 6802999000, VAT Rate: 13%
[2025-06-06 14:56:43] Tax ID: 6803001000, VAT Rate: 13%
[2025-06-06 14:56:43] Tax ID: 6803009000, VAT Rate: 13%
[2025-06-06 14:56:43] Tax ID: 6804100000, VAT Rate: 13%
[2025-06-06 14:56:43] Tax ID: 6804211000, VAT Rate: 13%
[2025-06-06 14:56:43] Tax ID: 6804219010, VAT Rate: 13%
[2025-06-06 14:56:43] Tax ID: 6804219090, VAT Rate: 13%
[2025-06-06 14:56:43] Tax ID: 6804221000, VAT Rate: 13%
[2025-06-06 14:56:43] Tax ID: 6804229010, VAT Rate: 13%
[2025-06-06 14:56:43] Tax ID: 6804229090, VAT Rate: 13%
[2025-06-06 14:56:43] Tax ID: 6804231000, VAT Rate: 1

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
